<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/Bingo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SERVIDOR

In [ ]:
%%writefile Servidor.py
import random
import sys
import multiprocessing.shared_memory
import time
import threading
import socket
import os

nuevoNumero = None
producir = True
maximos_jugadores = 5

# Objeto Condition para sincronizar la producción y el consumo
condicion = threading.Condition()

# Para compartir los numeros salientes uso MEMORIA COMPARTIDA
mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem", create=True, size=2)
mc.buf[0]= 0
mc.buf[1]= 0

# Función que produce números
def productor():
  global nuevoNumero
  global producir
  numeros = list(range(1, 91))   
  
  while producir:
    with condicion:
      # Esperar si hay un número disponible
      while nuevoNumero is not None:
        condicion.wait()    

      nuevoNumero = random.sample(numeros, 1)      
      numeros.remove(nuevoNumero[0])      
      
      if not numeros:      
        producir = False
            
      # Notificar al consumidor que hay un número disponible
      condicion.notify()

# Función que consume números
def consumidor():
  global nuevoNumero
  global mc  
  global cola_mensajes
  global producir
  global maximos_jugadores
  consumir = True
  bola = 1  
    
  # Para confirmar la recepcion de los datos por parte de los jugadores uso FIFO
  fifo_path = "./fifo"

  # Crear el FIFO si no existe
  if not os.path.exists(fifo_path):
    os.mkfifo(fifo_path, 0o600)

  # Abrir el FIFO en modo lectura
  fifo = os.open(fifo_path, os.O_RDONLY)
  
  while consumir:
    with condicion:
    # Esperar si no hay un número disponible
      while nuevoNumero is None:
        condicion.wait()         
             
      print("Bola numero %i: %i" %(bola, nuevoNumero[0])) 
      bola += 1  
      mc.buf[0] = nuevoNumero[0]  
              
      # Espero la recepcion de todos los jugadores
      recibidos = 0     
      
      while recibidos < maximos_jugadores:   
        mensaje = os.read(fifo, 1).decode()
        if mensaje == "":
          pass
        else:   
          recibidos += 1   
          
      time.sleep(0.5)
  
      nuevoNumero = None

      if not producir:
        consumir = False  

      if mc.buf[1] == 1:
        print("\nYa hay un ganador.")  
        consumir = False
        producir = False   
          
      # Notificar al productor que el número ha sido consumido
      condicion.notify()      

  mc.close()   
  os.close(fifo)  
  os.unlink(fifo_path)

#INICIO DEL MAIN
# Esperar a que se conecten los jugadores, uso sockets para notificar su conexion
jugadores_actuales = 0

# Crear un socket
ss = socket.socket()
ss.bind(("127.0.0.1", 2209))

# Escuchar conexiones entrantes
ss.listen(maximos_jugadores)

# Esperar a los jugadores
while jugadores_actuales < maximos_jugadores:
  # Aceptar una conexión entrante
  (cs, dir) = ss.accept()  

  # Incrementar el contador de conexiones
  jugadores_actuales += 1
  print("Se conceto el cliente numero: ", jugadores_actuales)

  # Enviar mensaje al Cliente
  mensaje = str(jugadores_actuales)
  cs.send(mensaje.encode())
  
# Cerrar el socket del servidor
cs.close()
ss.close()

print("\nInicia el bolillero:")

# Crear los hilos
hilo_productor = threading.Thread(target=productor)
hilo_consumidor = threading.Thread(target=consumidor)

# Iniciar los hilos
hilo_productor.start()
hilo_consumidor.start()

# Esperar a que ambos hilos terminen
hilo_productor.join()
hilo_consumidor.join()

mc.unlink()

Overwriting Servidor.py


JUGADOR

In [ ]:
%%writefile Jugador.py
import random
import sys
import multiprocessing.shared_memory
from multiprocessing.resource_tracker import unregister
import socket
import time
import os

def crear_carton():
  # Inicializar el cartón vacío
  carton = [[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0,0]]
  
  # Elegir 3 números aleatorios para cada columna del cartón
  for columna in range(9):
    if columna == 0:
      # Primer columna, números del 1 al 9
      numeros = list(range(1, 10))          
    elif columna == 8:       
      # Siguientes columnas, números del 80 al 90
      numeros = list(range(80, 91))
    else:
      # Resto de columnas, números del (columna * 10) al (columna * 10 + 9)
      numeros = list(range(columna * 10, columna * 10 + 10))
    
    seleccion = random.sample(numeros, 3)  
    for i in range(3):
      carton[i][columna] = min(seleccion)
      seleccion.remove(min(seleccion)) 

  # Dejar solo 15 valores (elimino 4 por fila)   
  for i in range(3):    
    numeros = list(range(0, 9))
    descarte = random.sample(numeros, 4)  
    for num in descarte:
      carton[i][num] = 0                 
  return carton

def imprimir_carton(carton):
  print("Cartón de Bingo:")
  for fila in carton:
    print(fila)

#INICIO DEL MAIN
# conectarse al servidor, uso de sockets
cs = socket.socket()
cs.connect(("127.0.0.1", 2209))

# Recibir mensaje del Servidor
mensaje = cs.recv(1024)
jugador = mensaje.decode()
print("Bienvenido, eres el jugador numero ", jugador)
print("")

# Cerrar el socket del cliente
cs.close()

# Crear un cartón de bingo
carton = crear_carton()

# Imprimir el cartón
imprimir_carton(carton)

# Paso valores a una lista
lista_numeros = [i for fila in carton for i in fila]

# Elimino los ceros de la lista
valor = 0
while valor in lista_numeros:
    lista_numeros.remove(valor)

# Ver numeros que van saliendo, uso memoria compartida
mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
unregister(mc._name, "shared_memory")

# Abro el fifo para confirmar recepcion de valores
fifo_path = "./fifo"

# Crear el FIFO si no existe
if not os.path.exists(fifo_path):
  os.mkfifo(fifo_path, 0o600)

# Abrir el FIFO en modo escritura
fifo = os.open(fifo_path, os.O_WRONLY)

mensaje = str(jugador)

numero = 0
bola = 1
consumir = True

print("\nEsperando a los demas jugadores.", end="")

while (mc.buf[0] == 0):
  pass

print("\nEstamos listo!!!\n\nSalio:")

while consumir:
  if mc.buf[1] == 1:
    print("\nFin del juego.")  
    print("Lo sentimos, ya hay un ganador.")      
    consumir = False
  else:
    nuevoNumero = mc.buf[0]  
    if nuevoNumero != numero:    
      numero = nuevoNumero
      print("Bola numero %i: %i" %(bola, numero))
      os.write(fifo, mensaje.encode()) # Notificar recepcion        
      bola += 1       
      if numero in lista_numeros:
        lista_numeros.remove(numero)    
      if not lista_numeros:
        print("\nFelicitaciones, ha ganado !!!")
        mc.buf[1] = 1
        consumir = False   

# Cierro la mc y fifo
mc.close()
os.close(fifo)

Overwriting Jugador.py


EJECUTAR (inicia al conectarse 5 jugadores)

In [ ]:
!nohup python Servidor.py 1>Servidor 2>errorSv &

In [ ]:
!nohup python Jugador.py 1>Jugador1 2>/dev/null &

In [ ]:
!nohup python Jugador.py 1>Jugador2 2>/dev/null &

In [ ]:
!nohup python Jugador.py 1>Jugador3 2>/dev/null &

In [ ]:
!nohup python Jugador.py 1>Jugador4 2>/dev/null &

In [ ]:
! python Jugador.py 

Bienvenido, eres el jugador numero  5

Cartón de Bingo:
[2, 10, 22, 0, 40, 53, 0, 0, 0]
[7, 16, 0, 0, 0, 54, 0, 74, 87]
[0, 0, 0, 39, 0, 58, 68, 79, 89]

Esperando a los demas jugadores.
Estamos listo!!!

Salio:
Bola numero 1: 75
Bola numero 2: 23
Bola numero 3: 41
Bola numero 4: 13
Bola numero 5: 69
Bola numero 6: 34
Bola numero 7: 15
Bola numero 8: 5
Bola numero 9: 9
Bola numero 10: 59
Bola numero 11: 83
Bola numero 12: 54
Bola numero 13: 27
Bola numero 14: 28
Bola numero 15: 51
Bola numero 16: 86
Bola numero 17: 35
Bola numero 18: 14
Bola numero 19: 24
Bola numero 20: 77
Bola numero 21: 78
Bola numero 22: 10
Bola numero 23: 84
Bola numero 24: 72
Bola numero 25: 1
Bola numero 26: 82
Bola numero 27: 8
Bola numero 28: 70
Bola numero 29: 55
Bola numero 30: 60
Bola numero 31: 68
Bola numero 32: 37
Bola numero 33: 29
Bola numero 34: 64
Bola numero 35: 45
Bola numero 36: 18
Bola numero 37: 50
Bola numero 38: 25
Bola numero 39: 79
Bola numero 40: 30
Bola numero 41: 53
Bola numero 42: 48
Bol

EMERGENCIA PARA CUANDO ALGO SE CERRO MAL

In [ ]:
import random
import sys
import multiprocessing.shared_memory
import time

mc = multiprocessing.shared_memory.SharedMemory(name="mi_mem")
mc.close()
mc.unlink()

In [ ]:
import os
fifo_path = "./fifo"
os.unlink(fifo_path)